In [5]:
import numpy as np

class AES():

    def __init__(self):
        self.mod_poly = 0x011B

    @staticmethod
    def byte_to_bits(byte):
        return np.array([(byte >> i) & 1 for i in reversed(range(8))], dtype=np.uint8)

    @staticmethod
    def bits_to_byte(bits):
        return sum((bits[i] << (7 - i)) for i in range(8))

    @staticmethod
    def bit_add(a, b):
        """Bitwise Addition"""
        return np.bitwise_xor(a, b)
    
    @staticmethod
    def bit_mul(a, b):
        """Bitwise Multiplication"""
        result = 0
        while b > 0:
            if b & 1:           # if the lowest bit of b is 1
                result ^= a     # add the current a to result
            a <<= 1             # multiply a by 2
            b >>= 1             # divide b by 2 (move to next bit)
        return result
    
    def bit_mod(self, poly):
        """Bitwise Polynomial Modulo"""
        mod_poly = self.mod_poly

        mod_len = mod_poly.bit_length()
        while poly.bit_length() >= mod_len:
            shift = poly.bit_length() - mod_len
            poly ^= mod_poly << shift  # XOR to subtract mod_poly shifted
        return poly
    
    def gf2_inverse(self, a):
        
        if a == 0:
            return 0x00

        u, v = a, self.mod_poly
        g1, g2 = 1, 0

        while u != 1:
            j = u.bit_length() - v.bit_length()
            if j < 0:
                u, v = v, u
                g1, g2 = g2, g1
                j = -j
            u ^= v << j
            g1 ^= g2 << j

        return self.bit_mod(g1)
        
    def affine_mapping(self, byte):
        # Define the 8×8 binary matrix A
        A = np.array([
            [1, 0, 0, 0, 1, 1, 1, 1],
            [1, 1, 0, 0, 0, 1, 1, 1],
            [1, 1, 1, 0, 0, 0, 1, 1],
            [1, 1, 1, 1, 0, 0, 0, 1],
            [1, 1, 1, 1, 1, 0, 0, 0],
            [0, 1, 1, 1, 1, 1, 0, 0],
            [0, 0, 1, 1, 1, 1, 1, 0],
            [0, 0, 0, 1, 1, 1, 1, 1]
        ])

        # Constant vector b
        b = np.array([0, 1, 1, 0, 0, 0, 1, 1])

        # Input byte as bit vector
        x = self.byte_to_bits(byte)

        # Perform affine transformation: y = A * x (mod 2) + b
        result_bits = np.array([
            np.bitwise_xor.reduce(A[i] & x) ^ b[i]
            for i in range(8)
        ], dtype=np.uint8)

        return self.bits_to_byte(result_bits)

    def SubBytes(self, byte):
        b_prime = self.gf2_inverse(byte)
        b = self.affine_mapping(b_prime)
        return b

    def test(self):
        assert self.bit_add(0x57, 0x83) == 0xd4, "Addition failed: 0x57 + 0x83 should be 0xd4"
        assert self.bit_mod(self.bit_mul(0x57, 0x83)) == 0xc1, "Multiplication failed. 0x57 * 0x83 should be 0xc1"

        print("All tests passed")


In [6]:
aes = AES()
aes.test()

All tests passed


In [7]:
aes.SubBytes(0xc2)

np.uint8(7)

In [17]:
from BitVector import *
c = BitVector(bitstring='01100011')
c.get_bitvector_in_hex()

'63'

In [15]:
0b01100011

99

# SubBytes
https://engineering.purdue.edu/kak/compsec/NewLectures/Lecture8.pdf